# 决策树

决策树其实表示的是给定特征条件下类的条件概率分布，主要包含3个步骤：

* 特征选择

* 决策树的生成

* 决策树的剪枝

在特征选择这一步骤，应当选取对训练数据有足够区分度的特征，能较好地把数据分开，通常特征选择的准则是信息增益或信息增益比

一般用信息熵来描述数据的不稳定性，信息熵越大，不稳定性越大

信息熵定义为：

$H(X) =  - \sum {p({x_i})} \log (p({x_i}))~~~~(i = 1,2, \ldots ,n)$

其中$X$表示的是随机变量，随机变量的取值为$({x_1},{x_2}, \ldots ,{x_n})$，p({x_i}) 表示事件发生的概率，且有$\sum {p({x_i})}  = 1$ 。信息熵的单位为bit。 对数一般以2为底或者以e为底

对于离散变量X,即为每种类别的概率值乘以概率值的对数，然后求和取负号

<font color = "Red">熵只依赖于X的分布，而与X的取值无关</font>

条件熵:它是条件分布上关于起条件作用的那个随机变量取平均之后的期望值。

$\begin{array}{l}
H(Y|X) &= \sum_{x \in X} {p(x)H(Y|X = x)} \\
 &=  - \sum_{x \in X} {p(x)} \sum_{y \in Y} {p(y|x)\log p(y|x)} \\
&=  - \sum_{x \in X} {\sum_{y \in Y} {p(x,y)\log p(y|x)} } 
\end{array}$

以书上的数据为例：

![avatar](img/decision_tree_1.png)

选取特征X = 年龄时：

&nbsp;&nbsp; p(x=青年)=5/15 , H(y|X=青年) = -( p(y=0|x=青年) * log(p(y=0|x=青年)) + p(y=1|x=青年) * log(p(y=1|xi=青年))) = -( 3/5 * log(3/5)  + 2/5 * log(2/5) ) 

&nbsp;&nbsp; p(x=中年)=5/15 , H(y|X=中年) = -( p(y=0|x=中年) * log(p(y=0|x=中年)) + p(y=1|x=中年) * log(p(y=1|xi=中年))) = -( 2/5 * log(2/5) + 3/5 * log(3/5) )

&nbsp;&nbsp; p(x=老年)=5/15 , H(y|X=老年) = -( p(y=0|x=老年) * log(p(y=0|x=老年)) + p(y=1|x=老年) * log(p(y=1|xi=老年))) = -( 1/5 * log(1/5) + 4/5 * log(4/5) )

H(Y|X=年龄) = p(x=青年) * H(y|x=青年) + p(x=中年) * H(y|x=中年) + p(x=老年) * H(y|x=老年)

信息增益描述了一个特征带来的信息量的多少，往往用于特征选择

信息增益 = 信息熵 - 条件熵

特征A对训练数据集D的信息增益g(D,A),定义为集合D的信息熵H（D）与特征A给定条件下D的经验条件熵H（D|A）之差，即

g(D,A) = H(D) - H(D|A)

一个特征往往会使一个随机变量Y的信息量减少，减少的部分就是信息增益

ID3算法:划分特征的标准是信息增益 g(D,A) = H(D) - H(D|A)

C4.5算法:划分特征的标准是信息增益比  g_r(D,A)= (H(D) - H(D|A))/H(D)

<table align="center">
    <tr bgcolor="#F6F8FA">
        <th>算法</th>
        <th>支持模型</th>
        <th>数结构</th>
        <th>特征选择</th>
        <th>连续值</th>
        <th>缺失值</th>
        <th>剪枝</th>
    </tr>
    <tr>
        <td>ID3</td>
        <td>分类</td>
        <td>多叉树</td>
        <td>信息增益</td>
        <td>不支持</td>
        <td>不支持</td>
        <td>不支持</td>
    </tr>
    <tr>
        <td>C4.5</td>
        <td>分类</td>
        <td>多叉树</td>
        <td>信息增益率</td>
        <td>支持</td>
        <td>支持</td>
        <td>支持</td>
    </tr>
    <tr>
        <td>CART</td>
        <td>分类、回归</td>
        <td>二叉树</td>
        <td>基尼系数</td>
        <td>支持</td>
        <td>支持</td>
        <td>支持</td>
    </tr>
</table>

In [150]:
import pandas as pd
import numpy as np
import json
from collections import Counter
data = [
    ["青年","否","否","一般","否"],
    ["青年","否","否","好","否"],
    ["青年","是","否","好","是"],
    ["青年","是","是","一般","是"],
    ["青年","否","否","一般","否"],
    ["中年","否","否","一般","否"],
    ["中年","否","否","好","否"],
    ["中年","是","是","好","是"],
    ["中年","否","是","非常好","是"],
    ["中年","否","是","非常好","是"],
    ["老年","否","是","非常好","是"],
    ["老年","否","是","好","是"],
    ["老年","是","否","好","是"],
    ["老年","是","否","非常好","是"],
    ["老年","否","否","一般","否"],
       ]
df = pd.DataFrame(data,columns = ["年龄","有工作","有自己的房子","信贷情况","类别"])
df

,年龄,有工作,有自己的房子,信贷情况,类别
0,青年,否,否,一般,否
1,青年,否,否,好,否
2,青年,是,否,好,是
3,青年,是,是,一般,是
4,青年,否,否,一般,否
5,中年,否,否,一般,否
6,中年,否,否,好,否
7,中年,是,是,好,是
8,中年,否,是,非常好,是
9,中年,否,是,非常好,是


In [155]:
class DecisionTree:
    
    # 得到一个数据集合的信息熵
    def get_entropy(self,df):
        # 所有label的取值
        label_list = df.iloc[:,-1].unique()
        
        # 计算H(Y)
        H_Y = 0
        for y_i in label_list:
            p_yi = len(df[df.get(df.columns[-1]) == y_i]) / len(df)
            H_Y += p_yi* np.log2(p_yi)
            
        H_Y *= -1
        return H_Y
    
    # 多数投票，当节点属性划分完后，还存在多个类别的样本，按多数投票决定（比如10个样本里虽然feature都一样，9个label是0,1个label是1的话，最后label被分为0）
    def majoritycnt(self,df):
        label_list = [i for i in df.iloc[:,-1]]
        counter = Counter(label_list)
        # most_common(k) 返回value最大的k个（key,value）列表,取出列表里的第一个元素，是一个(key,value)的tuple，再取key
        most_label = counter.most_common(1)[0][0]
        return most_label
    
    '''
    信息增益= 信息熵 - 条件熵
    取最大信息增益的feature作为划分的feature
    '''
    def get_splited_feature(self,df):
        # 先求原始集合的信息熵
        H_Y = self.get_entropy(df)
        # 存储最大的信息增益以及对应的feature
        best_feature = None
        best_gain = 0
        # 计算各个feature的条件熵，取信息增益最大的那个feature
        print("当前集合的feature：",df.columns[:-1])
        for index,feature_name in enumerate(df.columns[:-1]):
            # 这个feature的所有可能取值
            feature_set = df[feature_name].unique()
            # 计算按feature为A的划分之后的条件熵 H（D|A）= -∑p(xi)*H(y|x=xi)
            H_D_A = 0
            # 遍历这个feature所有可能取值
            for feature in feature_set:
                df_xi = df[df.get(feature_name)==feature]
                p_xi = len(df_xi) / len(df)
                # 条件熵 H（y|x=xi）
                h_y_xi = self.get_entropy(df_xi)
                H_D_A += p_xi * h_y_xi
            # 1.信息增益
            # 判断当前这个特征的信息增益是否比之前最好的大，如果是就更新
            cur_gain = H_Y - H_D_A
            print("按feature为'{}'划分的信息增益为{}".format(feature_name,cur_gain))
            
#             # 2.信息增益比
#             cur_gain = (H_Y - H_D_A) / H_Y
#             print("按feature为'{}'划分的信息增益比为{}".format(feature_name,cur_gain))
            
        
            if cur_gain > best_gain:
                best_gain = cur_gain
                best_feature = feature_name
        print("按feature={}进行划分\n".format(best_feature))
        return best_feature
    
    
    def fit(self, df):
        '''
            将每棵树的结构定义为一个json的形式{}
            
        '''
        def create_node(df, splited_value=None):
            # 如果集合中的标签只有一种了，则不需要继续往下分
            if len(df.iloc[:, -1].unique()) == 1:
                label = df.iloc[:, -1].unique()[0]
                return label
            #  如果集合中的属性划分完后，样本标签还不止一种的话，按投票多数表决
            elif len(df.iloc[0])==1:
                label = self.majoritycnt(df)
                return label
            # 继续往下递归划分
            else:
                #  最优的划分特征
                splited_feature = self.get_splited_feature(df)
                cur_node = {}
                son = {}
                #  遍历该特征的若干个不同取值，构建子树
                for feature_value in df[splited_feature].unique():
                    #  按splited_feature的取值划分为多颗子树
                    df_splited = df[df[splited_feature] == feature_value]
                    #  去除掉划分的特征那列
                    df_splited.pop(splited_feature)
                    # dic1.update(dic2) 利用dic2去更新dic1中的内容，如果key重复会被覆盖
                    son.update({feature_value:create_node(df_splited, feature_value)})
                cur_node[splited_feature] = son
                return cur_node

        self.root = create_node(df)
        print("====构建的决策树====")
        # json格式化展示
        data = json.dumps(root, sort_keys=True,ensure_ascii=False, indent=4, separators=(', ', ': '))
        print(data)
        return self.root
        
    def predict(self,test_data):
        # 预测
        print("====决策树预测====")
        print("预测样本:",test_data)
        cur_node = self.root
        # 字典类型表示一棵树，当不是字典类型的时候说明到了叶子节点
        while isinstance(cur_node,dict):
            # 当前节点划分的feature
            splited_feature = [i for i in cur_node.keys()][0]
            #  往下找到对应划分后的属性与测试数据属性一致的节点
            cur_node = cur_node.get(splited_feature).get(test_data[splited_feature])
        print("结果类别标签:",cur_node)

In [156]:
clf = DecisionTree()
root = clf.fit(df)
test_data = {"年龄":"老年","有工作":"否","有自己的房子":"否","信贷情况":"一般"}
clf.predict(test_data)

当前集合的feature： Index(['年龄', '有工作', '有自己的房子', '信贷情况'], dtype='object')
按feature为'年龄'划分的信息增益为0.08300749985576883
按feature为'有工作'划分的信息增益为0.32365019815155627
按feature为'有自己的房子'划分的信息增益为0.4199730940219749
按feature为'信贷情况'划分的信息增益为0.36298956253708536
按feature=有自己的房子进行划分

当前集合的feature： Index(['年龄', '有工作', '信贷情况'], dtype='object')
按feature为'年龄'划分的信息增益为0.2516291673878229
按feature为'有工作'划分的信息增益为0.9182958340544896
按feature为'信贷情况'划分的信息增益为0.47385138961004514
按feature=有工作进行划分

====构建的决策树====
{
    "有自己的房子": {
        "否": {
            "有工作": {
                "否": "否", 
                "是": "是"
            }
        }, 
        "是": "是"
    }
}
====决策树预测====
预测样本: {'年龄': '老年', '有工作': '否', '有自己的房子': '否', '信贷情况': '一般'}
结果类别标签: 否
